In [3]:
#!pip install torch torchvision numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 52.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 49.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 47.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 54.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 52.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 58.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 52.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 54.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
import random

import sys
sys.path.append('../src/utils')  # Adjust this path based on your directory structure

from po_fun import PO_util


In [4]:
def generate_synthetic_data(num_samples, n_nodes, K):
    """
    Generates synthetic partial order data and corresponding log(#LE) values.
    
    Parameters:
    - num_samples: Number of samples to generate.
    - n_nodes: Number of nodes in each partial order.
    - K: Number of dimensions for latent variables Z.
    
    Returns:
    - h_matrices: List of adjacency matrices representing partial orders.
    - log_LEs: List of log(#LE) values.
    """
    h_matrices = []
    log_LEs = []
    
    for _ in range(num_samples):
        # Randomly generate a DAG
        h = np.zeros((n_nodes, n_nodes), dtype=int)
        edges = []
        for i in range(n_nodes):
            for j in range(i+1, n_nodes):
                if random.random() < 0.3:  # 30% chance of an edge
                    h[i, j] = 1
                    edges.append((i, j))
        
        # Ensure transitivity (simple transitive closure)
        for k in range(n_nodes):
            for i in range(n_nodes):
                for j in range(n_nodes):
                    if h[i, k] and h[k, j]:
                        h[i, j] = 1
        
        h_matrices.append(h)
        
        # Estimate #LEs (for synthetic purposes, use a random value)
        # In practice, compute the exact or approximate number of linear extensions
        # Here, we use a random log_LE between 0 and 10
        log_LE = np.log(PO_util.nle(h))
        log_LEs.append(log_LE)
    
    return h_matrices, log_LEs


In [5]:
class PartialOrderDataset(Dataset):
    def __init__(self, h_matrices, log_LEs):
        """
        Initializes the dataset with partial order matrices and log(#LE) values.
        
        Parameters:
        - h_matrices: List of adjacency matrices (NumPy arrays).
        - log_LEs: List of corresponding log(#LE) values.
        """
        self.h_matrices = h_matrices
        self.log_LEs = log_LEs
    
    def __len__(self):
        return len(self.h_matrices)
    
    def __getitem__(self, idx):
        """
        Retrieves the h matrix and log_LE for a given index.
        
        Returns:
        - h_tensor: Tensor representation of the h matrix.
        - log_LE: Tensor representation of log(#LE).
        """
        h = self.h_matrices[idx]
        log_LE = self.log_LEs[idx]
        
        # Flatten the h matrix and convert to float tensor
        h_tensor = torch.tensor(h, dtype=torch.float32).flatten()
        log_LE = torch.tensor(log_LE, dtype=torch.float32)
        
        return h_tensor, log_LE


In [6]:
# Parameters
NUM_SAMPLES = 1000    # Total number of samples
N_NODES = 10          # Number of nodes in each partial order
K_DIM = 5             # Number of dimensions for latent variables Z (not directly used here)

# Generate synthetic data
h_matrices, log_LEs = generate_synthetic_data(NUM_SAMPLES, N_NODES, K_DIM)

# Split into training and testing sets (80% train, 20% test)
split_idx = int(0.8 * NUM_SAMPLES)
train_h = h_matrices[:split_idx]
train_log_LE = log_LEs[:split_idx]
test_h = h_matrices[split_idx:]
test_log_LE = log_LEs[split_idx:]

# Create Dataset objects
train_dataset = PartialOrderDataset(train_h, train_log_LE)
test_dataset = PartialOrderDataset(test_h, test_log_LE)

# Create DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
class LE_Predictor(nn.Module):
    def __init__(self, input_size, embedding_dim=128):
        """
        Initializes the neural network.
        
        Parameters:
        - input_size: Size of the input vector (flattened h matrix).
        - embedding_dim: Size of the embedding layer.
        """
        super(LE_Predictor, self).__init__()
        
        self.embedding = nn.Linear(input_size, embedding_dim)
        self.relu1 = nn.ReLU()
        self.fc1 = nn.Linear(embedding_dim, 64)
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.output = nn.Linear(32, 1)  # Predicting a single scalar value
        
    def forward(self, x):
        """
        Defines the forward pass.
        
        Parameters:
        - x: Input tensor.
        
        Returns:
        - Output tensor.
        """
        x = self.embedding(x)
        x = self.relu1(x)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        x = self.relu3(x)
        x = self.output(x)
        return x.squeeze()  # Remove unnecessary dimensions


In [ ]:
# Determine input size
INPUT_SIZE = N_NODES * N_NODES  # e.g., 10x10 = 100

# Initialize the model
model = LE_Predictor(input_size=INPUT_SIZE, embedding_dim=128)

# Define the loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Define the optimizer (Adam optimizer)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Number of epochs
NUM_EPOCHS = 50

# Device configuration (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (h_batch, log_LE_batch) in enumerate(train_loader):
        h_batch = h_batch.to(device)
        log_LE_batch = log_LE_batch.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(h_batch)
        
        # Compute loss
        loss = criterion(outputs, log_LE_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Calculate average loss over the epoch
    avg_loss = running_loss / len(train_loader)
    
    # Evaluate on the test set
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for h_batch, log_LE_batch in test_loader:
            h_batch = h_batch.to(device)
            log_LE_batch = log_LE_batch.to(device)
            outputs = model(h_batch)
            loss = criterion(outputs, log_LE_batch)
            test_loss += loss.item()
    avg_test_loss = test_loss / len(test_loader)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], "
          f"Train Loss: {avg_loss:.4f}, "
          f"Test Loss: {avg_test_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Function to evaluate the model and collect predictions
def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for h_batch, log_LE_batch in data_loader:
            h_batch = h_batch.to(device)
            log_LE_batch = log_LE_batch.to(device)
            outputs = model(h_batch)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(log_LE_batch.cpu().numpy())
    return predictions, actuals

# Get predictions and actuals for the test set
predictions, actuals = evaluate_model(model, test_loader, device)

# Convert to NumPy arrays for easier handling
predictions = np.array(predictions)
actuals = np.array(actuals)

# Compute evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print(f"Test MSE: {mse:.4f}")
print(f"Test R² Score: {r2:.4f}")

# Plotting Predicted vs Actual
plt.figure(figsize=(8,6))
plt.scatter(actuals, predictions, alpha=0.7)
plt.xlabel("Actual log(#LE)")
plt.ylabel("Predicted log(#LE)")
plt.title("Actual vs Predicted log(#LE)")
plt.plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], 'r--')  # Diagonal line
plt.show()
